# Analysing FAF data

## Intro

Hi. I'm `teolicy`, a [FAF] [player] and hobbyist data geek. In this notebook I'll present some analyses I've done on FAF data compiled `api.faforever.com` and parsed replay blobs.

[FAF]: http://www.faforever.com
[player]: http://api.faforever.com/data/player/368866

I hope to achieve two things with this:

* Show the community what analyses are possible and build interest in future work
* Source ideas from the community on what analyses people would like to see

This notebook assumes you know what is FAF and its concepts, and you mainly want to see some analyses. The [Kaggle dataset] is a better starting point if you're not a FAF player but mainly interested in the dataset from an analytical perspective.

[Kaggle dataset]: https://www.kaggle.com/datasets/yanivaknin/fafdata

## Quickstart

Let's start by looking at this panel of scatter plots trying to identify behaviours of strong players; click to zoom in. 

[![Behaviour by Rating and Map size](https://user-images.githubusercontent.com/101657/177016397-5a47fe4d-862a-489b-8610-3e28487ea06c.png)](https://user-images.githubusercontent.com/101657/177016397-5a47fe4d-862a-489b-8610-3e28487ea06c.png)

How do we read this?
* This is every valid 1v1 ladder game from Feb'16 to May'22 (~650K games total)
* Every game creates two dots on the chart: one for each player
* Each of the three columns represents the map size: `5x5`, `10x10`, and `20x20`
* Each of the nine rows represents a different behaviour to analyse
* Each X-axis shows the rating of the player; Y-axis shows the behaviour

The behaviours are:

 Behaviour | Description 
 --------- | ----------- 
 Actions Per Minute       | Counting only `issue` and `factory_issue` commands (i.e., telling units to do something or telling factories to do something)
 Reclaim | Percentage of issue actions with command type `Reclaim` (reclaim not through patrol/attack move)
 Defense | Percentage of build actions creating units with the [`DEFENSE` category] (point defense, shield, etc)
 Overcharge | Percentage of issue actions being `Overcharge`
 Transport | Percentage issue actions of the `Transport*` family (load, unload, ferry, etc)
 Tn Ratio | Percentage of build actions creating units at each tech level (remember 'build one Mantis and set factory to repeat' is just one T1 action)

[`DEFENSE` category]: https://github.com/FAForever/fa/blob/02a0db58b444b66f9e58f9fdb3a9c6c53fb8fa39/units/UAB2101/UAB2101_unit.bp#L38

These behaviours were extracted from each replay file for each of the ~650K games. And we can extract a lot more: I should find time to write what we can (and can't) get out of replay files, it's a bit unintuitive how some easy things are hard and some complex things are easy or at least doable.

From this panel of scatterplots, we can get some insights:
* Clearly stronger players have higher APM and do more reclaim
* It could be argued stronger players use defensive units less; needs more work
* Stronger players are most consistent using transports in bigger maps (and
  less in smaller maps)

I've created a [Datastudio Dashboard](https://datastudio.google.com/reporting/ad26e447-e1fd-4856-b7d0-78447dfcfde7/page/rFzvC) in which you can slice and dice the data yourself a bit. I find Datastudio quite limiting (I made the chart above using [Tableau](https://tableau.com) which isn't free), but I thought it's better than nothing.

### What I need from you

This is one of many possible analyses we can run, and the Datastudio dashboard is limited to a specific set of metrics. If you have cool questions you'd like answered, please file or comment on an [issue](https://github.com/yaniv-aknin/fafanalysis/issues). I'll try to respond to these, and where I'm able to I'll create an analysis and publish the findings.

If you're a data scientist or otherwise analytically trained and want to get involved or advise me: great! Find me on [Discord or Zulip](https://forum.faforever.com/topic/251/who-owns-faf-and-how-is-the-organisation-structured), and you can read more about the underlying data on the [Kaggle page] and in the remainder of this notebook.

[Kaggle page]: https://www.kaggle.com/datasets/yanivaknin/fafdata

## Hello data

Here's a crash course on the data I extracted and how I can query it.
* If you're not too interested in the technical details you can stop reading now (but please go file [ideas](https://github.com/yaniv-aknin/fafanalysis/issues) so I have stuff to analyse!).
* If you *are* interested, please note that you should be able to run this Colab notebook yourself. My [BigQuery datasets are public](https://cloud.google.com/bigquery/public-data#share_a_dataset_with_the_public), and if you have a Google Cloud Platform account, everything Should Just Work.

In [1]:
# BigQuery authorization and boilerplate
from google.colab import auth
auth.authenticate_user()
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15, 10)

In [10]:
q = """
SELECT * FROM fafalytics.faf_ladder_1v1.issue_cmds TABLESAMPLE SYSTEM (10 PERCENT) WHERE DATE(startTime) > "2022-05-01" LIMIT 100
"""
df = pd.read_gbq(q, project_id='fafalytics')
df

/usr/local/lib/python3.7/dist-packages/google/cloud/bigquery/table.py:1582: FutureWarning: Data is timezone-aware. Converting timezone-aware data to timezone-naive by passing dtype='datetime64[ns]' to DataFrame or Series is deprecated and will raise in a future version. Use `pd.Series(values).dt.tz_localize(None)` instead.
  bqstorage_client=bqstorage_client, dtypes=dtypes


,id,startTime,offset_ms,player,type,command_type,units_number,blueprint,position
0,16937791,2022-05-02 01:58:36+00:00,257100,1,issue,7,1,ura0102,[]
1,16937791,2022-05-02 01:58:36+00:00,354400,1,issue,7,1,ura0101,[]
2,16937791,2022-05-02 01:58:36+00:00,652700,1,issue,7,1,url0202,[]
3,16937791,2022-05-02 01:58:36+00:00,794400,1,issue,7,1,ura0102,[]
4,16937474,2022-05-02 00:43:56+00:00,130900,1,issue,7,1,xsl0201,[]
...,...,...,...,...,...,...,...,...,...
95,17032941,2022-05-13 22:41:26+00:00,372800,1,issue,7,1,uel0104,[]
96,17032941,2022-05-13 22:41:26+00:00,447100,1,issue,7,1,uel0103,[]
97,17032941,2022-05-13 22:41:26+00:00,452100,1,issue,7,1,uel0103,[]
98,17032941,2022-05-13 22:41:26+00:00,515100,1,issue,7,1,uel0103,[]


The table you see is the basic unit I'm working with. A typical row looks like this:

|index|id|startTime|offset\_ms|player|type|command\_type|units\_number|blueprint|position|
|---|---|---|---|---|---|---|---|---|---|
|0|16937791|2022-05-02 01:58:36+00:00|257100|1|issue|7|1|ura0102||
|1|16937791|2022-05-02 01:58:36+00:00|354400|1|issue|7|1|ura0101||

And it says something like "in game with id `16937791`, which started on 2nd May'22, 257.1 seconds into the game, player with in-game id `1` sent command of type 7 to one unit, associated with blueprint `ura0102`.

Cryptic, huh?
* It's a bit tricky, but in-game id's can be resolved to FAF player ids; I resolved all games and ```SELECT faf_player_id, login FROM `fafalytics.faf_ladder_1v1.metadata` WHERE id = "16937791" AND replay_player_id = 1``` will tell us the player was [JadedJason84](http://api.faforever.com/data/player/44479).
* Command type `7` is `BuildFactory`, telling a factory to build something. `ura0102` is simply an [intercepter](https://unitdb.faforever.com/?id=Ura0102). We say this was sent to "one unit" because presumably only one air factory was selected.

This is fairly straightforward, only we have about 10M games, and a thousand or so instructions per player per game (this varies a fair bit). Whitling this down a bit to ladder 1v1 games 2016-2022, we're still left with almost a billion rows and over 50GB.

So that's the job really. If you look at the `faf_ladder_1v1.issue_stats` view, you'll see how it calculates the behaviours you saw earlier in the chart.

In [12]:
from google.cloud import bigquery
client = bigquery.Client()

view = client.get_table("fafalytics.faf_ladder_1v1.issue_stats")
print(view.view_query)

SELECT
  c.id AS game_id,
  c.startTime as start_time,
  SAFE_DIVIDE(COUNT(*), MAX(offset_ms)/1000/60) AS issue_apm,

  SAFE_DIVIDE(COUNTIF(faf_ladder_1v1.COMMAND_NAME(c.command_type) = "Reclaim"), COUNT(*)) AS reclaim_ratio,
  SAFE_DIVIDE(COUNTIF(faf_ladder_1v1.COMMAND_NAME(c.command_type) = "OverCharge"), COUNT(*)) AS overcharge_ratio,
  SAFE_DIVIDE(COUNTIF(faf_ladder_1v1.COMMAND_NAME(c.command_type) LIKE "Transport%"), COUNT(*)) AS transport_ratio,

  SAFE_DIVIDE(COUNTIF(u.structure), COUNTIF(blueprint IS NOT NULL)) AS structure_ratio,
  SAFE_DIVIDE(COUNTIF(u.dimension = "air"), COUNTIF(blueprint IS NOT NULL)) AS air_ratio,
  SAFE_DIVIDE(COUNTIF(u.dimension = "land"), COUNTIF(blueprint IS NOT NULL)) AS land_ratio,
  SAFE_DIVIDE(COUNTIF(u.dimension = "naval"), COUNTIF(blueprint IS NOT NULL)) AS naval_ratio,
  SAFE_DIVIDE(COUNTIF(u.tech = 1), COUNTIF(blueprint IS NOT NULL)) AS t1_ratio,
  SAFE_DIVIDE(COUNTIF(u.tech = 2), COUNTIF(blueprint IS NOT NULL)) AS t2_ratio,
  SAFE_DIVIDE(COUNT

We should (collectively) think about additional behaviours we'd like to extract this way, or additional metadata I should get out of the replay (e.g., not only `issue` commands, get the notifications that are sent like `Tech 2 Air HQ upgrade done!`, etc.